# TPOT Demo

In [1]:
!pip install TPOT

    100% |████████████████████████████████| 901kB 109kB/s ta 0:00:01
    100% |████████████████████████████████| 942kB 204kB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 355kB/s ta 0:00:01
  Running setup.py bdist_wheel for TPOT ... done
  Stored in directory: /Users/ykatada/Library/Caches/pip/wheels/86/5c/dd/c7673fbaccb901ec1a4eb79017fa5b65766805d2a98f954b9a
  Running setup.py bdist_wheel for deap ... done
  Stored in directory: /Users/ykatada/Library/Caches/pip/wheels/22/ea/bf/dc7c8a2262025a0ab5da9ef02282c198be88902791ca0c6658
  Running setup.py bdist_wheel for stopit ... done
  Stored in directory: /Users/ykatada/Library/Caches/pip/wheels/3c/85/2b/2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7
Successfully built TPOT deap stopit
You are using pip version 9.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import numpy as np
import pandas as pd  
from sklearn.model_selection import train_test_split  
from sklearn.impute import SimpleImputer
from tpot import TPOTRegressor
from collections import Counter

ModuleNotFoundError: No module named 'sklearn.impute'

In [100]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [101]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [118]:
exclude_cols = ["Id", "SalePrice"]
x = [colnames for colnames in train.columns if colnames not in exclude_cols]

In [119]:
def check_type(x):
    if type(x) == float or type(x) == int:
        return True
    else:
        return False

In [120]:
num_cols = []
cat_cols= []
for each_col in x:
    a = Counter(train[each_col])
    if all(list(map(check_type, list(a.keys())))):
        num_cols.append(each_col)
    else:
        cat_cols.append(each_col)

In [121]:
num_imputer = SimpleImputer(strategy="median")
cat_imputer = SimpleImputer(strategy="most_frequent")

In [122]:
num_X_train = num_imputer.fit_transform(train[num_cols])
cat_X_train = cat_imputer.fit_transform(train[cat_cols])
X_train = np.c_[num_X_train, cat_X_train]
X_train = pd.DataFrame(X_train, columns=num_cols+cat_cols)

num_X_test = num_imputer.transform(test[num_cols])
cat_X_test = cat_imputer.transform(test[cat_cols])
X_test = np.c_[num_X_test, cat_X_test]
X_test= pd.DataFrame(X_test, columns=num_cols+cat_cols)

In [123]:
X_train.shape

(1460, 79)

In [124]:
whole = X_train.append(X_test)

In [125]:
whole = pd.get_dummies(whole, columns=cat_cols)
X_train = np.array(whole.iloc[:X_train.shape[0]], dtype=np.float)
X_test = np.array(whole.iloc[X_train.shape[0]:],  dtype=np.float)

In [126]:
y = "SalePrice"

In [127]:
y_train = train[y]

In [130]:
my_tpot = TPOTRegressor(generations=10, verbosity=2) 

In [131]:
my_tpot.fit(X_train, y_train)

HBox(children=(IntProgress(value=0, description='Optimization Progress', max=1100, style=ProgressStyle(description_width='initial')), HTML(value='')))



TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: RandomForestRegressor(LassoLarsCV(input_matrix, normalize=False), bootstrap=True, max_features=0.7000000000000001, min_samples_leaf=4, min_samples_split=16, n_estimators=100)


TPOTRegressor(config_dict=None, crossover_rate=0.1, cv=5,
       disable_update_check=False, early_stop=None, generations=10,
       max_eval_time_mins=5, max_time_mins=None, memory=None,
       mutation_rate=0.9, n_jobs=1, offspring_size=None,
       periodic_checkpoint_folder=None, population_size=100,
       random_state=None, scoring=None, subsample=1.0, use_dask=False,
       verbosity=2, warm_start=False)

In [134]:
y_pred = my_tpot.predict(X_test)

In [137]:
test["SalePrice"] = y_pred

In [138]:
test[["Id", "SalePrice"]].to_csv("TPOT_submission.csv", index=False)

<img src="TPOT_kaggle_submission.png"></img>